In [1]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import BinaryRelevance
from xgboost import XGBClassifier

In [2]:
os.getcwd()

'C:\\Users\\fanfan\\Documents\\Capstone\\DSCI_591_capstone-BCStats\\notebooks'

In [3]:
os.chdir('../')

In [4]:
os.getcwd()

'C:\\Users\\fanfan\\Documents\\Capstone\\DSCI_591_capstone-BCStats'

In [5]:
from src.models.eval import theme_results

In [6]:
df_combined_train = pd.read_csv("data/interim/qual_combined_train.csv",index_col=0)
#df_combined_train['clean_comments'] = df_combined_train['comments'].apply(preprocess)

In [9]:
df_combined_train.head()

,comments,CPD,CB,EWC,Exec,FWE,SP,RE,Sup,SW,TEPE,VMG,OTH
0,I believe the funding the ministry receives is...,0,0,0,0,0,0,0,0,1,0,0,0
1,"I would appreciate a less chaotic, fear driven...",0,0,1,0,0,0,0,0,1,1,1,0
2,Wage increases cost of living lift freezes. ...,0,1,0,0,0,0,0,0,0,0,0,0
3,Walk the talk with respect to honesty and inte...,0,0,1,0,0,0,1,0,0,0,0,0
4,We require more funding for more FTEs.,0,0,0,0,0,0,0,0,1,0,0,0


### Balance the train dataset

In [7]:
x = np.array(df_combined_train.comments)
y = np.array(df_combined_train.loc[:,"CPD":"OTH"])

In [8]:
counts = df_combined_train.sum(axis = 0)[1:13]

In [9]:
counts

CPD     3346
CB      4157
EWC     2487
Exec    3227
FWE     1309
SP      2619
RE      2410
Sup     3472
SW      5027
TEPE    5457
VMG     3758
OTH      629
dtype: object

In [10]:
sample_counts = [max(counts) - i for i in counts]

In [11]:
sample_counts

[2111, 1300, 2970, 2230, 4148, 2838, 3047, 1985, 430, 0, 1699, 4828]

In [12]:
labels = list(df_combined_train.columns)[1:13]
df_balanced = df_combined_train

for index, label in enumerate(labels):
    df_balanced = df_balanced.append(df_combined_train[df_combined_train[label]==1].sample(n=sample_counts[index],
                                                             replace = True))

In [13]:
df_balanced.sum(axis = 0)

comments    I believe the funding the ministry receives is...
CPD                                                      8339
CB                                                       8634
EWC                                                      8033
Exec                                                     8390
FWE                                                      6163
SP                                                       7586
RE                                                       7877
Sup                                                      8995
SW                                                       9574
TEPE                                                     8685
VMG                                                      8093
OTH                                                      5757
dtype: object

In [14]:
train, validation = train_test_split(df_balanced, random_state=42, test_size=0.30, shuffle=True)

In [15]:
x_train = np.array(train['comments'])
x_valid = np.array(validation['comments'])
y_train = np.array(train.loc[:,"CPD":"OTH"])
y_valid = np.array(validation.loc[:,"CPD":"OTH"])

In [16]:
x_train

array(['I have repeatedly asked for a new laptop, it has been approved and ordered, but I still have not received it.   Also, I feel strongly that at the Executive Director, ADM, and DM level there is no meaningful communication with our Program. I also feel that my manager has very little direct involvement with their staff and rely too heavily on the chain of command to pass on important information.   When our last Director was terminated, there was no support for those who felt negatively impacted by their departure. There was no real explanation.   ',
       "I would like my organization to focus on reducing stress levels.   This could be done by ensuring the person I report to has the knowledge base needed to make well founded decisions and to be able to provide assistance and guidance to the work unit.   Although the person I report to has been in the position for multiple years they are unable to provide direct feedback that can help me do my job better therefore the feedback I

In [18]:
y_train

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,4), 
                              stop_words="english", max_features=15000)

In [21]:
x_train = vectorizer.fit_transform(x_train)
x_valid = vectorizer.transform(x_valid)

In [23]:
xgb = XGBClassifier()
classifier = BinaryRelevance(classifier=xgb)
classifier.fit(x_train,y_train)
prediction = classifier.predict(x_valid).toarray()

OSError: [WinError -529697949] Windows Error 0xe06d7363

In [24]:
from sklearn.linear_model import LogisticRegression
classifier = BinaryRelevance(classifier=LogisticRegression())
classifier.fit(x_train,y_train)
prediction = classifier.predict(x_valid).toarray()

C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to sile

In [25]:
theme_results(y_valid,prediction)

Overall Accuracy: 0.7496 
Hamming Loss: 0.0286 
Hamming Loss (pred. zeros): 0.1536


,Label,Y_proportion,Pred_proportion,Accuarcy,Precision,Recall
0,CPD,0.159857,0.153483,0.974122,0.936462,0.899123
1,CB,0.165721,0.157881,0.983747,0.973355,0.927308
2,EWC,0.154631,0.142074,0.971126,0.942575,0.866035
3,Exec,0.164510,0.153802,0.961374,0.909242,0.850058
4,FWE,0.116961,0.118809,0.991395,0.956009,0.971117
5,SP,0.143668,0.139779,0.977628,0.933881,0.908607
6,RE,0.148639,0.142202,0.970744,0.919767,0.879931
7,Sup,0.173242,0.161451,0.961820,0.918279,0.855776
8,SW,0.182739,0.161642,0.948435,0.905757,0.801186
9,TEPE,0.163618,0.149786,0.972274,0.953617,0.873004


In [26]:
from sklearn.ensemble import GradientBoostingClassifier
classifier = BinaryRelevance(classifier=GradientBoostingClassifier())
classifier.fit(x_train,y_train)
prediction = classifier.predict(x_valid).toarray()

MemoryError: 

In [27]:
categories = list(df_combined_train.columns.values)
categories = categories[1:]
categories

['CPD',
 'CB',
 'EWC',
 'Exec',
 'FWE',
 'SP',
 'RE',
 'Sup',
 'SW',
 'TEPE',
 'VMG',
 'OTH']

In [30]:
xgb = XGBClassifier()

prediction_df_xgb = pd.DataFrame()

for category in categories:
        
    # Training Gradient radient Tree Boosting classifier model on train data
    xgb.fit(x_train, train[category])
    
    # calculating test accuracy
    prediction = xgb.predict(x_valid)
    prediction_df_xgb[category] = prediction
   

In [34]:
prediction_xgb = np.array(prediction_df_xgb)

In [35]:
theme_results(y_valid,prediction_xgb)

Overall Accuracy: 0.2834 
Hamming Loss: 0.0913 
Hamming Loss (pred. zeros): 0.1536


,Label,Y_proportion,Pred_proportion,Accuarcy,Precision,Recall
0,CPD,0.159857,0.133342,0.922621,0.809273,0.675040
1,CB,0.165721,0.140162,0.948435,0.907231,0.767308
2,EWC,0.154631,0.064695,0.894002,0.875862,0.366447
3,Exec,0.164510,0.084518,0.881382,0.771493,0.396358
4,FWE,0.116961,0.089936,0.960227,0.929128,0.714441
5,SP,0.143668,0.093123,0.924979,0.868583,0.562999
6,RE,0.148639,0.079992,0.905348,0.837450,0.450686
7,Sup,0.173242,0.104851,0.892090,0.811550,0.491170
8,SW,0.182739,0.090955,0.877111,0.829012,0.412626
9,TEPE,0.163618,0.088725,0.906113,0.892960,0.484223
